In [ ]:
from pybeam.datamodels import nodes_from_csv, elements_from_csv
from pybeam.assembly import assemble_system_matrices, reindex_dof
from pybeam.simulation import simulate
from pybeam.modal_parameters import get_modal_parameters
from pybeam._utilities import pprint_array
from pybeam.plotting import plot_structure, plot_modeshape, set_style, plot_deformations
from pybeam.fatigue import get_stress_history
import matplotlib.pyplot as plt

%matplotlib qt5
set_style()
import numpy as np

import logging

logging.getLogger("pybeam").setLevel(logging.INFO)

### Initialize system matrices

Assemble the system matrices.

The damping matrix is defined by Rayleigh's damping model. Change coefficients as needed.

In [ ]:
nodes = nodes_from_csv("data/frame/nodes.csv")
elements = elements_from_csv("data/frame/elements.csv", nodes)

stiffness, mass = assemble_system_matrices(elements)
damping = 0.01 * mass + 0.001 * stiffness
# damping = None

elements = reindex_dof(elements)

plt.figure()
plot_structure(elements, node_labels=True, element_labels=True)

### Set simulation parameters

In [ ]:
# Number of DOF
n_dof = stiffness.shape[0]

# Time step size
dt = 0.005

# End time
t_end = 100
# Time steps
time = np.arange(start=0, stop=t_end + dt, step=dt)

# Loading
loads = np.zeros((n_dof, len(time)))
loads[9, :] = 100 * np.cos(4 * time)

# Initial conditions
x_0 = np.zeros((n_dof,))
# x_0[16] = 0.5
v_0 = np.zeros((n_dof,))

# Integration parameter
beta = 1 / 4

### Simulate the time response

In [ ]:
x, v, a = simulate(
    stiffness=stiffness,
    mass=mass,
    damping=damping,
    initial_disp=x_0,
    initial_vel=v_0,
    loads=loads,
    time=time,
    beta=beta,
    gamma=1 / 2,  # Don't change this unless you know what you're doing
)

### Get stress history at node

The `get_stress_history` function takes a displacement time series, defined in global coordinates and defned for all DOF, an element, the distance away from the element's neutral axis and the node to be used (can be `"start"` or `"end"`).

The output is an N-sized vector of normal stresses, where N is then number of time steps.

In [ ]:
stresses = get_stress_history(x, elements[10], z=0.005, node="end")

### Plot stress history

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
ax = plt.gca()

# Adjust the look of the plot
ax.set_xlabel("Time")
ax.set_ylabel("Stress")
ax.set_xlim(0, time[-1])

ax.plot(time, stresses * 1e-6, color="white")
plt.show()

### Rainflow counting

In [ ]:
import rainflow

scaling_factor = 1e-6  # To go from Pa to MPa

bins = rainflow.count_cycles(stresses, ndigits=0)
ranges = [bin[0] * scaling_factor for bin in bins]
counts = [bin[1] for bin in bins]

### Plot cycle counts

In [ ]:
plt.figure()
ax = plt.gca()
ax.set_xlabel("Stress range")
ax.set_ylabel("Count")
plt.hist(ranges, bins=20, edgecolor="black", color="gray")  # Change the number of bars in the plot
plt.show()

### Define SN curve

In [ ]:
from pybeam.datamodels import SnCurveDnv, SnCurveEurocode
import numpy as np

# We define an SN curve with detail category 100
sn_curve = SnCurveEurocode(d_sigma_c=100)

# Get the fatigue damage for each stress range
damages = []

# We iterate over the ranges and counts
for i, (stress_range, n_i) in enumerate(zip(ranges, counts)):
    # N_i is the fatigue life at the current stress range
    N_i = sn_curve.get_cycles_at_range(stress_range)

    # If N_i is infinite, we don't need to do anything
    if N_i == np.inf:
        continue
    # Otherwise, we calculate the damage according to Palmgren-Miner's rule
    else:
        damage = n_i / N_i
        damages.append(damage)

print(f"The total damage is: {sum(damages)}")